# Localization System

## Table of Content 
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<!-- <li><a href="#eda">Exploratory Data Analysis</a></li> -->
<li><a href="#model"> Building a model </a></li>
</ul>

<a id='intro'></a>
## Introduction 
Localziation systems built using machine learning classifiers, data is collected at Cairo University Faculty of 
engineering Biomedical Engineering Department. The Data is a collection of WiFi\`s strenghts (dbm) of 3 Wifi Networks available in the department. Identifying a patient\`s location in a hospital is useful for many reasons one of which is to identify points of conjestions and try to rearrange the hospital survices, also used to identify how many patients are in the hospital. 

So We are going to dive into our gathered data and invetigate our findings. 

In [1]:
# Basic Importations 
import pandas as pd 
import numpy as np

<a id="#wrangling"></a>
## Data Wrangling 
Let\`s dive into our gathered data and find it\`s secerets. 

In [2]:
# Read our datasets 
df_ta = pd.read_csv('esp_csv_only/csv/Ta.csv')
df_lab = pd.read_csv('esp_csv_only/csv/Lab.csv')

# Show Heads 
df_lab.describe()

,StudBME1,STUDBME2,SBME_STAFF3,SBME_STAFF,CUFE,RehabLab,lab001,BMEStudentLab3,CMP_LAB,CMP_LAB1,CMP_LAB2
count,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.0,149.0,149.000000,149.000000
mean,-81.624161,-57.053691,-87.214765,-77.194631,-68.946309,-87.946309,-84.315436,0.0,0.0,-58.570470,-60.677852
std,12.310442,4.658304,2.255815,27.815919,35.480826,2.189611,4.304440,0.0,0.0,30.234948,31.482356
min,-89.000000,-73.000000,-93.000000,-89.000000,-90.000000,-94.000000,-92.000000,0.0,0.0,-85.000000,-87.000000
25%,-85.000000,-60.000000,-89.000000,-88.000000,-88.000000,-89.000000,-87.000000,0.0,0.0,-74.000000,-77.000000
50%,-84.000000,-58.000000,-87.000000,-87.000000,-87.000000,-88.000000,-85.000000,0.0,0.0,-73.000000,-74.000000
75%,-82.000000,-54.000000,-86.000000,-86.000000,-85.000000,-87.000000,-82.000000,0.0,0.0,-71.000000,-71.000000
max,0.000000,-47.000000,-80.000000,0.000000,0.000000,-78.000000,-71.000000,0.0,0.0,0.000000,0.000000


In [3]:
df_ta.describe()

,StudBME1,STUDBME2,SBME_STAFF3,SBME_STAFF,CUFE,RehabLab,lab001,BMEStudentLab3,CMP_LAB,CMP_LAB1,CMP_LAB2
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.0,151.0,151.000000,151.000000
mean,-78.980132,-69.589404,-71.913907,-84.940397,-83.238411,-68.847682,-86.317881,0.0,0.0,-77.874172,-75.317881
std,10.841568,9.522795,35.969420,10.426397,19.977557,37.952909,10.471785,0.0,0.0,20.161781,25.352021
min,-92.000000,-85.000000,-95.000000,-95.000000,-91.000000,-93.000000,-95.000000,0.0,0.0,-90.000000,-89.000000
25%,-84.000000,-73.000000,-91.000000,-88.000000,-91.000000,-90.000000,-89.000000,0.0,0.0,-86.500000,-86.000000
50%,-80.000000,-70.000000,-91.000000,-86.000000,-87.000000,-90.000000,-88.000000,0.0,0.0,-82.000000,-83.000000
75%,-76.000000,-67.000000,-86.000000,-85.000000,-85.000000,-86.500000,-86.000000,0.0,0.0,-80.000000,-81.000000
max,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000


The first look into the data insights the following:

- BMEStudentLab3 & CMP_LAB are out of range and their values wont benefit our model. 
- There are a lot of zeros in the recorded data.

### The Zeros problem :
The data is recorded in a way that, if one of the specified wifi networks is not found or out of range it records the value for that moment as zero, according to the documentation of the ESP Module the strenghts of a wifi network is a value that ranges from -100 to -50 dB, -100 for bad network connectivity (Signal Strength) and -50 good network connnectivity.
<img src="sources/wifi.jpeg">
So we can treat a zero value as miss recorded value (seems to be an error in the chip) and replace it with any method we want, we will stick with replacing with the mean for now. 

## Imputing miss calculated records 

In [4]:
# Drop the two bad coloumns 
bad_cols = [df_lab.BMEStudentLab3.name, df_lab.CMP_LAB.name]
df_lab.drop(bad_cols, axis=1,inplace=True)
df_ta.drop(bad_cols, axis=1, inplace=True)

In [5]:
# Replace Zeros with the mean
cols = df_lab.columns
df_lab[cols] = df_lab[cols].replace({0:np.nan})
df_ta[cols] = df_ta[cols].replace({0:np.nan})

In [6]:
df_lab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   StudBME1     146 non-null    float64
 1   STUDBME2     149 non-null    float64
 2   SBME_STAFF3  149 non-null    float64
 3   SBME_STAFF   132 non-null    float64
 4   CUFE         118 non-null    float64
 5   RehabLab     149 non-null    float64
 6   lab001       149 non-null    float64
 7   CMP_LAB1     118 non-null    float64
 8   CMP_LAB2     118 non-null    float64
dtypes: float64(9)
memory usage: 10.6 KB


In [7]:
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   StudBME1     149 non-null    float64
 1   STUDBME2     149 non-null    float64
 2   SBME_STAFF3  121 non-null    float64
 3   SBME_STAFF   149 non-null    float64
 4   CUFE         143 non-null    float64
 5   RehabLab     116 non-null    float64
 6   lab001       149 non-null    float64
 7   CMP_LAB1     142 non-null    float64
 8   CMP_LAB2     136 non-null    float64
dtypes: float64(9)
memory usage: 10.7 KB


In [8]:
# Find missing in coloumns and replace with mean
miss_lab = []
miss_ta = []

for i in df_lab.columns:
    missing_lab = df_lab[i].isnull().sum()
    missing_ta = df_ta[i].isnull().sum()
    if missing_lab >0 :
        miss_lab.append(i)
    if missing_ta >0:
        miss_ta.append(i)
print(miss_lab)
print(miss_ta)

['StudBME1', 'SBME_STAFF', 'CUFE', 'CMP_LAB1', 'CMP_LAB2']
['StudBME1', 'STUDBME2', 'SBME_STAFF3', 'SBME_STAFF', 'CUFE', 'RehabLab', 'lab001', 'CMP_LAB1', 'CMP_LAB2']


In [9]:
# Replace missing with mean 
for col_lab, col_ta in zip(miss_lab, miss_ta):
    df_lab[col_lab].fillna(df_lab[col_lab].mean(), inplace=True)
    df_ta[col_ta].fillna(df_ta[col_ta].mean(), inplace=True)


In [10]:
df_lab.describe()

,StudBME1,STUDBME2,SBME_STAFF3,SBME_STAFF,CUFE,RehabLab,lab001,CMP_LAB1,CMP_LAB2
count,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000
mean,-83.301370,-57.053691,-87.214765,-87.136364,-87.059322,-87.946309,-84.315436,-73.957627,-76.618644
std,3.704216,4.658304,2.255815,1.044760,1.274979,2.189611,4.304440,2.613360,4.165365
min,-89.000000,-73.000000,-93.000000,-89.000000,-90.000000,-94.000000,-92.000000,-85.000000,-87.000000
25%,-85.000000,-60.000000,-89.000000,-88.000000,-88.000000,-89.000000,-87.000000,-74.000000,-77.000000
50%,-84.000000,-58.000000,-87.000000,-87.136364,-87.000000,-88.000000,-85.000000,-73.000000,-76.000000
75%,-82.000000,-54.000000,-86.000000,-86.000000,-87.000000,-87.000000,-82.000000,-73.000000,-74.000000
max,-73.000000,-47.000000,-80.000000,-85.000000,-84.000000,-78.000000,-71.000000,-69.000000,-69.000000


In [11]:
df_ta.describe()

,StudBME1,STUDBME2,SBME_STAFF3,SBME_STAFF,CUFE,RehabLab,lab001,CMP_LAB1,CMP_LAB2
count,151.000000,151.000000,151.000000,151.000000,151.000000,116.000000,149.000000,142.000000,136.000000
mean,-80.040268,-70.523490,-89.743802,-86.080537,-87.895105,-89.620690,-87.476510,-82.809859,-83.625000
std,5.766431,5.024717,1.741757,3.349660,2.983763,0.965988,3.017032,4.563085,3.782881
min,-92.000000,-85.000000,-95.000000,-95.000000,-91.000000,-93.000000,-95.000000,-90.000000,-89.000000
25%,-84.000000,-73.000000,-91.000000,-88.000000,-91.000000,-90.000000,-89.000000,-87.000000,-86.000000
50%,-80.040268,-70.523490,-91.000000,-86.000000,-87.895105,-90.000000,-88.000000,-83.000000,-85.000000
75%,-76.500000,-68.000000,-88.000000,-85.000000,-85.000000,-90.000000,-86.000000,-80.000000,-82.000000
max,-62.000000,-54.000000,-86.000000,-71.000000,-82.000000,-86.000000,-78.000000,-69.000000,-70.000000


In [ ]:
# Set Classes for each location
df_ta['location'] = 1
df_lab['location']= 2

<a id='model'></a>
## Building the Model 

In [ ]:
# Importations 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_auc_score

In [ ]:
# iris = datasets.load_iris()
# X = iris['data']
# y = iris['target']

In [ ]:
pd.DataFrame(X)

In [ ]:
classifiers = {
    'knn': KNeighborsClassifier(5),
    'NB': GaussianNB(),
    'tree': DecisionTreeClassifier(max_depth=5),
    'forest': RandomForestClassifier(n_estimators=10, max_depth=5),
    'SV': SVC(probability=True),
    'LR': LogisticRegression(solver='newton-cg')
}

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
pd.Series(y_train).value_counts()

In [ ]:
# Metrics 
metrics = ['accuracy', 'f1_weighted', 'roc_auc_ovr', 'precision_weighted', 'recall_weighted']

# Cross Validate the models 
scores = {}
for name, clf in classifiers.items():
    print(f'Training {name} model .. ')
    res = cross_validate(clf, X_train, y_train, cv=10, scoring=metrics)
    
    score = {"test_f1_weighted": np.mean(res['test_f1_weighted']), 
             'test_roc_auc_ovr': np.mean(res['test_roc_auc_ovr']), 
             'test_precision_weighted': np.mean(res['test_precision_weighted']),
             'test_recall_weighted': np.mean(res['test_recall_weighted']),
             'test_accuracy': np.mean(res['test_accuracy'])
            }
    
    print("test_f1_weighted", score['test_f1_weighted'])
    print('test_roc_auc_ovr', score['test_roc_auc_ovr'])
    print('test_precicison', score['test_precision_weighted'])
    print('test_recall', score['test_recall_weighted'])
    print('accuracy', score['test_accuracy'])
    print('\n')
    
    # Add each model`s scores to scores
    scores[name] = score

In [ ]:
models = []
thresholds = {"test_f1_weighted": 97.0, 
              'test_roc_auc_ovr': 97.0,
              'test_precision_weighted': 97.0,
              'test_recall_weighted': 97.0,
              'test_accuracy': 97.0}

for clf, results in scores.items():
    pass